### Importação de bibliotecas
Carrega os pacotes necessários para regressão, avaliação de modelos e tuning de hiperparâmetros com Optuna.

In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score, mean_absolute_error
from sklearn.model_selection import cross_val_score
import optuna

### Leitura dos dados tratados
Importa o dataset previamente limpo e transformado durante a etapa de EDA, permitindo prosseguir diretamente com a construção do modelo.

In [3]:
df = pd.read_csv('../Data/powerconsumption_after_EDA.csv')
df.head()

,Hour,Temperature,Humidity,WindSpeed,GeneralDiffuseFlows,DiffuseFlows,PowerConsumption_Zone1,PowerConsumption_Zone2,PowerConsumption_Zone3,TotalConsumption
0,0,6.559,73.8,0.083,0.051,0.119,34055.69620,16128.87538,20240.96386,70425.53544
1,0,6.414,74.5,0.083,0.070,0.085,29814.68354,19375.07599,20131.08434,69320.84387
2,0,6.313,74.5,0.080,0.062,0.100,29128.10127,19006.68693,19668.43373,67803.22193
3,0,6.121,75.0,0.083,0.091,0.096,28228.86076,18361.09422,18899.27711,65489.23209
4,0,5.921,75.7,0.081,0.048,0.085,27335.69620,17872.34043,18442.40964,63650.44627


### Preparação do conjunto de treino e teste
Separa o target (`TotalConsumption`) das variáveis preditoras e realiza a divisão dos dados em treino e teste para validação independente do modelo.

In [4]:
X = df.drop(['PowerConsumption_Zone1',
             'PowerConsumption_Zone2',
             'PowerConsumption_Zone3',
             'TotalConsumption'], axis=1)

y = df['TotalConsumption']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Função objetivo para otimização com Optuna

Define a função de avaliação a ser maximizada durante a busca bayesiana de hiperparâmetros com Optuna. O espaço de busca inclui parâmetros-chave do Random Forest, como profundidade da árvore, número de estimadores e critérios de divisão.

A métrica utilizada é o coeficiente R², avaliado por validação cruzada com 5 folds.


In [5]:
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'max_depth': trial.suggest_int('max_depth', 5, 30),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 20),
        'max_features': trial.suggest_categorical('max_features', [None, 'sqrt', 'log2']),
    }

    model = RandomForestRegressor(**params, random_state=42)
    
    score = cross_val_score(model, X_train, y_train, scoring='r2', cv=5, n_jobs=-1).mean()
    return score

### Otimização de hiperparâmetros
Inicia o processo de tuning com Optuna, utilizando uma função objetivo para encontrar a melhor combinação de hiperparâmetros.

In [6]:
optuna.logging.set_verbosity(optuna.logging.WARNING)
study = optuna.create_study(direction='maximize')
study.optimize(objective, timeout = 3600)

### Extração dos melhores hiperparâmetros
Recupera os parâmetros ideais definidos pelo processo de otimização, que serão usados para refinar o modelo final.

In [7]:
print("Best hyperparameters found:")
print(study.best_params)
print("Best mean R2:", study.best_value)

Best hyperparameters found:
{'n_estimators': 181, 'max_depth': 23, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': None}
Best mean R2: 0.9346719836151536


### Treinamento do modelo
Treina um modelo de regressão baseado em Random Forest. Essa técnica é adequada para capturar relações não lineares e reduzir risco de overfitting.

In [8]:
best_model = RandomForestRegressor(**study.best_params, random_state=42)
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)

### Avaliação do desempenho do modelo

Calcula-se um conjunto abrangente de métricas de regressão para quantificar a performance do modelo sobre o conjunto de teste:

- **MSE (Erro Quadrático Médio)**: penaliza mais fortemente grandes erros.
- **MAE (Erro Absoluto Médio)**: métrica robusta a outliers.
- **R² (Coeficiente de Determinação)**: representa a proporção da variância explicada.
- **MAPE**: erro percentual médio, útil para interpretação em escala relativa.
- **nMSE (MSE normalizado)**: mede o erro em relação à média ao quadrado da variável dependente, facilitando comparação entre modelos.

A apresentação dos resultados em valores absolutos e percentuais fornece uma análise mais intuitiva e comparável da qualidade preditiva do modelo.


In [9]:
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

nMSE_percent = (mse / np.mean(y_test)**2)
mape = mean_absolute_percentage_error(y_test, y_pred)

print(f"MSE: {mse:.4f}")
print(f"MAE: {mae:.4f}")
print(f"R2: {r2:.4f}")
print('-'*10)
print(f"MSE in percentage: {nMSE_percent*100:.2f}%")
print(f"MAE in percentage: {mape*100:.2f}%")
print(f"R2 in percentage: {r2*100:.2f}%")

MSE: 14769335.3208
MAE: 2596.9510
R2: 0.9405
----------
MSE in percentage: 0.30%
MAE in percentage: 3.81%
R2 in percentage: 94.05%
